In [1]:
import pandas as pd
import numpy as np
import os
import time
from collections import defaultdict
from sklearn import linear_model
from sklearn.metrics import accuracy_score,f1_score,recall_score,roc_curve
from sklearn import metrics
import seaborn as sns
import matplotlib.pyplot as plt

**Combine the symptoms and the demographic information**

In [2]:
directory = "../Data/With_Improved_Target/With_Demographics/"

In [3]:
def read_file(filename):
    temp = pd.read_csv(directory+filename)
    temp['index'] = range(0,temp.shape[0])
    temp.set_index('index',inplace = True)
    return temp

In [4]:
def get_all_files(files_):
    data = defaultdict()
    for i in files_:
        print("Reading file : ",i)
        data[i[:-4]] = read_file(i)
    return data

In [5]:
files_ = ['nyumc.csv','goviral.csv','fluwatch.csv','hongkong.csv','hutterite.csv']

In [6]:
data = get_all_files(files_)

Reading file :  nyumc.csv
Reading file :  goviral.csv
Reading file :  fluwatch.csv
Reading file :  hongkong.csv
Reading file :  hutterite.csv


In [7]:
print("The read files are : ",data.keys())

The read files are :  dict_keys(['nyumc', 'goviral', 'fluwatch', 'hongkong', 'hutterite'])


*separate the symptoms and demographics information*

In [8]:
symptoms = ['fever','sorethroat','cough','muscle','headache','fatigue','vomit','nausea','diarrhea','chills',
            'sneeze','shortness of breath','phlegm','blockednose','earache','leg pain','runnynose']

demographic = ['age 0-4','age 5-15','age 16-44','age 45-64','age 65+','male','female']


In [9]:
def combine(symptoms,demographic):
    combined_features = [(p1, p2) for p1 in symptoms for p2 in demographic]
    return combined_features

In [10]:
combined_features = combine(symptoms,demographic)

In [11]:
def create_columns(combined_features):
    columns_ = [i[0]+"_"+i[1] for i in combined_features]
    return columns_

In [12]:
columns_ = create_columns(combined_features)

In [13]:
def create_new_dataframe(data_,combined_features,columns_):
    new_data = pd.DataFrame(columns=columns_)
    for i in combined_features:
        symp = i[0]
        dem  = i[1]
        new_data[symp+"_"+dem] = np.where((data_[symp] == 1) & (data_[dem] == 1), 1, 0)
    new_data['virus'] = data_['virus']
    return new_data
    
    

In [14]:
data_nyumc = create_new_dataframe(data['nyumc'],combined_features,columns_)
print("Original number of sample points : ",data['nyumc'].shape)
print("Final number of sample points : ",data_nyumc.shape)

Original number of sample points :  (21907, 25)
Final number of sample points :  (21907, 120)


In [15]:
data_nyumc.head()

,fever_age 0-4,fever_age 5-15,fever_age 16-44,fever_age 45-64,fever_age 65+,fever_male,fever_female,sorethroat_age 0-4,sorethroat_age 5-15,sorethroat_age 16-44,...,leg pain_male,leg pain_female,runnynose_age 0-4,runnynose_age 5-15,runnynose_age 16-44,runnynose_age 45-64,runnynose_age 65+,runnynose_male,runnynose_female,virus
0,0,0,1,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2,0,1,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,1
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,1,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [16]:
data_gv = create_new_dataframe(data['goviral'],combined_features,columns_)
print("Original number of sample points : ",data['goviral'].shape)
print("Final number of sample points : ",data_gv.shape)

Original number of sample points :  (520, 25)
Final number of sample points :  (520, 120)


In [17]:
data_gv.head()

,fever_age 0-4,fever_age 5-15,fever_age 16-44,fever_age 45-64,fever_age 65+,fever_male,fever_female,sorethroat_age 0-4,sorethroat_age 5-15,sorethroat_age 16-44,...,leg pain_male,leg pain_female,runnynose_age 0-4,runnynose_age 5-15,runnynose_age 16-44,runnynose_age 45-64,runnynose_age 65+,runnynose_male,runnynose_female,virus
0,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,1,0,0,1,0,0
1,0,0,0,0,0,0,0,0,0,1,...,1,0,0,0,0,0,0,0,0,1
2,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,1,0,0,0,1,1
3,0,0,0,0,1,1,0,0,0,0,...,1,0,0,0,0,0,1,1,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,1


In [18]:
data_fw = create_new_dataframe(data['fluwatch'],combined_features,columns_)
print("Original number of sample points : ",data['fluwatch'].shape)
print("Final number of sample points : ",data_fw.shape)

Original number of sample points :  (915, 25)
Final number of sample points :  (915, 120)


In [19]:
data_fw.head()

,fever_age 0-4,fever_age 5-15,fever_age 16-44,fever_age 45-64,fever_age 65+,fever_male,fever_female,sorethroat_age 0-4,sorethroat_age 5-15,sorethroat_age 16-44,...,leg pain_male,leg pain_female,runnynose_age 0-4,runnynose_age 5-15,runnynose_age 16-44,runnynose_age 45-64,runnynose_age 65+,runnynose_male,runnynose_female,virus
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,1,1
1,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,1,0,0,0,1,0
2,0,0,0,1,0,0,1,0,0,0,...,0,0,0,0,0,1,0,0,1,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
4,0,0,0,1,0,0,1,0,0,0,...,0,0,0,0,0,1,0,0,1,0


In [20]:
data_hk = create_new_dataframe(data['hongkong'],combined_features,columns_)
print("Original number of sample points : ",data['hongkong'].shape)
print("Final number of sample points :",data_hk.shape)

Original number of sample points :  (4954, 25)
Final number of sample points : (4954, 120)


In [21]:
data_hk.head()

,fever_age 0-4,fever_age 5-15,fever_age 16-44,fever_age 45-64,fever_age 65+,fever_male,fever_female,sorethroat_age 0-4,sorethroat_age 5-15,sorethroat_age 16-44,...,leg pain_male,leg pain_female,runnynose_age 0-4,runnynose_age 5-15,runnynose_age 16-44,runnynose_age 45-64,runnynose_age 65+,runnynose_male,runnynose_female,virus
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0


In [22]:
data_ht = create_new_dataframe(data['hutterite'],combined_features,columns_)
print("Original number of sample points : ",data['hutterite'].shape)
print("Final number of sample points : ",data_ht.shape)

Original number of sample points :  (1281, 25)
Final number of sample points :  (1281, 120)


In [23]:
data_ht.head()

,fever_age 0-4,fever_age 5-15,fever_age 16-44,fever_age 45-64,fever_age 65+,fever_male,fever_female,sorethroat_age 0-4,sorethroat_age 5-15,sorethroat_age 16-44,...,leg pain_male,leg pain_female,runnynose_age 0-4,runnynose_age 5-15,runnynose_age 16-44,runnynose_age 45-64,runnynose_age 65+,runnynose_male,runnynose_female,virus
0,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,1
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,1,0,1
3,0,1,0,0,0,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,1
4,0,0,0,0,0,0,0,0,1,0,...,0,0,0,1,0,0,0,0,1,1


#### Save the new data files with combined symptoms and demographics

In [24]:
def save_file(dataframe_,filename_):
    print(len(list(dataframe_.columns)))
    directory = "../Data/With_Improved_Target/Combined/"
    filename = directory+filename_
    if os.path.isfile(filename):
        print("File exists!")
    else:
        dataframe_.to_csv(filename,index=False)

In [26]:
save_file(data_nyumc,"nyumc.csv")

120


In [27]:
save_file(data_gv,"goviral.csv")

120


In [28]:
save_file(data_gv,"fluwatch.csv")

120


In [29]:
save_file(data_hk,"hongkong.csv")

120


In [30]:
save_file(data_ht,"hutterite.csv")

120
